In [1]:
from JHProgress import JHProgress
import pandas as pd

In [2]:
jhp=JHProgress()

2025-11-30 20:54:10,718 - INFO - Tickers downloaded successfully
2025-11-30 20:54:12,065 - INFO - Data for AAPL fetched successfully from SEC
2025-11-30 20:54:12,103 - INFO - Caracteristica EarningsPerShareDiluted obtenida para AAPL
2025-11-30 20:54:13,511 - INFO - Data for AAPL fetched successfully from SEC
2025-11-30 20:54:13,565 - INFO - Caracteristica NetIncomeLoss obtenida para AAPL
2025-11-30 20:54:14,255 - INFO - Data for AAPL fetched successfully from SEC
2025-11-30 20:54:14,307 - INFO - Caracteristica OperatingIncomeLoss obtenida para AAPL
2025-11-30 20:54:15,014 - INFO - Data for AAPL fetched successfully from SEC
2025-11-30 20:54:15,076 - INFO - Caracteristica Revenues obtenida para AAPL
2025-11-30 20:54:16,073 - INFO - Data for AAPL fetched successfully from SEC
2025-11-30 20:54:16,115 - INFO - Caracteristica GrossProfit obtenida para AAPL
2025-11-30 20:54:17,052 - INFO - Data for AAPL fetched successfully from SEC
2025-11-30 20:54:17,105 - INFO - Caracteristica CostsAndExp

In [3]:
jhp.df_fundamentals

,label,description,units.USD/shares.start,units.USD/shares.end,units.USD/shares.val,units.USD/shares.accn,units.USD/shares.fy,units.USD/shares.fp,units.USD/shares.form,units.USD/shares.filed,...,Metrica,units.USD.start,units.USD.end,units.USD.val,units.USD.accn,units.USD.fy,units.USD.fp,units.USD.form,units.USD.filed,units.USD.frame
0,"Earnings Per Share, Diluted",The amount of net income (loss) for the period...,2006-10-01,2007-09-29,3.93,0001193125-09-214859,2009.0,FY,10-K,2009-10-27,...,EarningsPerShareDiluted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Earnings Per Share, Diluted",The amount of net income (loss) for the period...,2006-10-01,2007-09-29,3.93,0001193125-10-012091,2009.0,FY,10-K/A,2010-01-25,...,EarningsPerShareDiluted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Earnings Per Share, Diluted",The amount of net income (loss) for the period...,2007-09-30,2008-06-28,4.10,0001193125-09-153165,2009.0,Q3,10-Q,2009-07-22,...,EarningsPerShareDiluted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Earnings Per Share, Diluted",The amount of net income (loss) for the period...,2008-03-30,2008-06-28,1.19,0001193125-09-153165,2009.0,Q3,10-Q,2009-07-22,...,EarningsPerShareDiluted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Earnings Per Share, Diluted",The amount of net income (loss) for the period...,2007-09-30,2008-09-27,5.36,0001193125-09-214859,2009.0,FY,10-K,2009-10-27,...,EarningsPerShareDiluted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13042,Liabilities,Sum of the carrying amounts as of the balance ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Liabilities,NaN,2024-12-31,210537000.0,0001662991-25-000215,2025.0,Q2,10-Q,2025-08-08,NaN
13043,Liabilities,Sum of the carrying amounts as of the balance ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Liabilities,NaN,2024-12-31,210537000.0,0001662991-25-000257,2025.0,Q3,10-Q,2025-11-06,CY2024Q4I
13044,Liabilities,Sum of the carrying amounts as of the balance ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Liabilities,NaN,2025-03-31,172312000.0,0001662991-25-000085,2025.0,Q1,10-Q,2025-05-08,CY2025Q1I
13045,Liabilities,Sum of the carrying amounts as of the balance ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Liabilities,NaN,2025-06-30,220154000.0,0001662991-25-000215,2025.0,Q2,10-Q,2025-08-08,CY2025Q2I
